In [168]:
import numpy as np
from collections import Counter

In [169]:
def loadDataset():
    postingList = [
        ["my", "dog", "has", "flea", "problems", "help", "please"],
        ["maybe", "not", "take", "him", "to", "dog", "park", "stupid"],
        ["my", "dalmation", "is", "so", "cute", "I", "love", "him"],
        ["stop", "posting", "stupid", "worthless", "garbage"],
        ["mr", "licks", "ate", "my", "steak", "how", "to", "stop", "him"],
        ["quit", "buying", "worthless", "dog", "food", "stupid"]]
    classVec = [0, 1, 0, 1, 0, 1]
    return postingList, classVec    

In [170]:
def createVocabList(dataset):
    Vocavset = set()
    for line in dataset:
        for word in line:
            Vocavset.add(word)
    return list(Vocavset)

In [171]:
def SetOfWord2Vec(vocablist, inputset):
    returnvec = [0]*len(vocablist)
    for word in inputset:
        if word in vocablist:
            returnvec[vocablist.index(word)] = 1
        else:
            print ("the word: %s is not in vocabulary !" %word)
    return returnvec

In [172]:
postingList, classVec = loadDataset()
vocablist = createVocabList(postingList)


In [173]:
returnvec = SetOfWord2Vec(vocablist, postingList[0])
returnvec

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1]

In [174]:
trainMatrix = []
for postingdoc in postingList:
    trainMatrix.append(SetOfWord2Vec(vocablist, postingdoc))

In [175]:
def trainNB0(trainMatrix, traincategory):
    num_traindoc = len(trainMatrix)
    nums_words = len(trainMatrix[0])
    pAbusive = sum(traincategory)/float(num_traindoc)
    p0_num, p1_num = np.ones(nums_words), np.ones(nums_words)
    p0_Denom, p1_Denom = 2.0, 2.0
    for i in range(num_traindoc):
        if traincategory[i] == 1:
            p1_num += trainMatrix[i]
            p1_Denom += sum(trainMatrix[i])
        else:
            p0_num += trainMatrix[i]
            p0_Denom += sum(trainMatrix[i])
    # p1vec = p1_num/p1_Denom
    # p0vec = p0_num/p0_Denom
    p1vec = np.log(p1_num/p1_Denom)
    p0vec = np.log(p0_num/p0_Denom)
    return p0vec, p1vec, pAbusive

    

In [176]:
traincategory = classVec
p0vec, p1vec, pAbusive = trainNB0(trainMatrix, traincategory)
pAbusive

0.5

In [177]:
p1vec

array([-1.94591015, -2.35137526, -3.04452244, -2.35137526, -2.35137526,
       -3.04452244, -3.04452244, -3.04452244, -3.04452244, -2.35137526,
       -3.04452244, -2.35137526, -3.04452244, -3.04452244, -3.04452244,
       -2.35137526, -2.35137526, -3.04452244, -3.04452244, -3.04452244,
       -1.65822808, -2.35137526, -2.35137526, -1.94591015, -2.35137526,
       -3.04452244, -3.04452244, -2.35137526, -3.04452244, -3.04452244,
       -2.35137526, -3.04452244])

In [178]:
p0vec

array([-3.25809654, -3.25809654, -2.56494936, -3.25809654, -2.56494936,
       -2.56494936, -2.56494936, -2.56494936, -2.56494936, -3.25809654,
       -2.56494936, -2.15948425, -2.56494936, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -3.25809654, -3.25809654, -3.25809654, -2.56494936, -3.25809654,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -3.25809654, -1.87180218])

In [179]:
def classifyNB(vec2classify, p0vec, p1vec, pclass1):
    p1 = sum(vec2classify*p1vec) + np.log(pclass1)
    p0 = sum(vec2classify*p0vec) + np.log(1-pclass1)
    return 1 if p1 > p0 else 0 

In [180]:
def testingNB(testEntry):
    postingList, classVec = loadDataset()
    vocablist = createVocabList(postingList)
    trainMatrix = []
    for postingdoc in postingList:
        trainMatrix.append(SetOfWord2Vec(vocablist, postingdoc))
    p0vec, p1vec, pAbusive = trainNB0(np.array(trainMatrix), np.array(classVec))
    thisdoc = np.array(SetOfWord2Vec(vocablist, testEntry))
    print(testEntry, "classify as:", classifyNB(thisdoc, p0vec, p1vec, pAbusive))
    

    

In [181]:
testEntry = ["love", "my", "dalmation"]
testingNB(testEntry)
testEntry = ["stupid", "garbage"]
testingNB(testEntry)

['love', 'my', 'dalmation'] classify as: 0
['stupid', 'garbage'] classify as: 1


In [182]:
def BagOfWord2Vec(vocablist, inputset):
    returnvec = [0]*len(vocablist)
    for word in inputset:
        if word in vocablist:
            returnvec[vocablist.index(word)] += 1
        else:
            print ("the word: %s is not in vocabulary !" %word)
    return returnvec

In [183]:
mysent = "This book is the best book on Python or M.L. I have ever laid eyes upon."
mysent1 = mysent.split(" ")
mysent1

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

In [184]:
import re
regEX = re.compile('\\W+') #匹配非英文字母和数字
listoftokens = regEX.split(mysent)
listoftokens

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon',
 '']

In [185]:
ListOfTokens = [token.lower() for token in listoftokens if len(token)>0]
ListOfTokens

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']